In [1]:
import pyspark
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

import os


jar_packages = [
  "io.delta:delta-spark_2.12:3.2.0"
]

spark_extensions = [
  "io.delta.sql.DeltaSparkSessionExtension"
]

spark = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
        #.master("spark://spark-master:7077")
		.config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000")
        .config("hive.metastore.uris", "thrift://metastore:9083")
        .config("spark.sql.warehouse.dir", "user/delta/warehouse")
        .enableHiveSupport()
        .getOrCreate()
)


# spark = configure_spark_with_delta_pip(builder)
# .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
# .config("spark.hadoop.fs.s3a.access.key", "SEU_ACCESS_KEY") \
# .config("spark.hadoop.fs.s3a.secret.key", "SEU_SECRET_KEY") \
# .config("spark.hadoop.fs.s3a.endpoint", "http://SEU_MINIO_ENDPOINT:9000") \
# .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    
print("Spark Running")

spark.sql("SHOW DATABASES").show()
spark.sql("CREATE OR REPLACE TABLE my_table (id INT, name STRING) USING DELTA")
spark.sql("INSERT INTO my_table VALUES (1, 'Alice'), (2, 'Bob')")
spark.sql("SELECT * FROM my_table").show()

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-82d6360f-68f9-426b-b879-23a5008dbb41;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 336ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |  

Spark Running
+---------+
|namespace|
+---------+
|  default|
+---------+



24/10/17 03:41:42 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`my_table` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
24/10/17 03:41:43 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/10/17 03:41:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



In [10]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
%sql trino://trino@trino:8080

 * trino://trino@trino:8080
Done.


Catalog
hive
system


In [13]:
%%sql
select * FROM hive.test_db.my_table

 * trino://trino@trino:8080
(trino.exceptions.TrinoExternalError) TrinoExternalError(type=EXTERNAL, name=UNSUPPORTED_TABLE_TYPE, message="Cannot query Delta Lake table 'test_db.my_table'", query_id=20240909_033231_00005_heity)
[SQL: select * FROM hive.test_db.my_table]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [25]:
%%sql
CREATE TABLE hive.test_db.my_table2 (id INT, name char) WITH (format='parquet')

 * trino://trino@trino:8080
(trino.exceptions.TrinoExternalError) TrinoExternalError(type=EXTERNAL, name=HIVE_METASTORE_ERROR, message="Required field 'engine' is unset! Struct:TableStatsRequest(dbName:test_db, tblName:my_table2, colNames:[id, name], engine:null)", query_id=20240909_034823_00017_heity)
[SQL: CREATE TABLE hive.test_db.my_table2 (id INT, name char) WITH (format='parquet')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
